# MODELO DE RECOMENDACIONES EN SPOTITFY Y TRANSFERENCIA A DEEZER

### Pasos

1. Seguimos las indicaciones del recomendador de Spotify del principio del fichero
2. Realizamos los pasos para transferir la playlist con las recomendaciones, los cuales se encuentrarn al final del codigo



# Recomendador con la integración de la API de Spotify

## 1: Crear un Spotify Authorization Token

Debido a las restricciones de la API de Spotify, es necesario añadir el SpotifyForDevelopers a tu cuenta de Spotify para poder ejecutar este código. Ya que no es posible inicar sesión directamente en tu cuenta, con estos tokens podemos validar las credenciales y obterner la información desde Spotify.

### Pasos

1. Accede a https://developer.spotify.com/dashboard/ e inicia sesión (o resgistrate si aún no tienes cuenta).  
2. Accede a  https://developer.spotify.com/console/post-playlists/ y pulsa en "Get Token".
3. Selecciona los siguientes casillas:  
    a. playlist-modify-public  
    b. playlist-modify-private  
    c. user-read-recently-played  
4. Presiona en "Request Token" y "Agree". Ahora deberías ver un valor debajo de **OAuth Token**. Copialo, va a ser necesario en el futuro.  
5. Ahora, abre Spotify. Abra el menú desplegable junto a su nombre en la esquina superior derecha, y selecciona **Account**.  
6. Copie el **Username** para usarlo en el futuro.  
7. En la siguiente celda, pegue los valores obtenidos anteriormente en los campos **auth_token** y **user_id**. 


**Nota: Los tokens de Spotify caducan cada hora , por lo que será necesario que siga de nuevo los pasos detallados si alguna de las celdas falla.**


In [ ]:
import os
os.system("pip3 install spotipy ")
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import requests

# inicalizar variaables de credenciales de acceso
auth_token = ""
user_id = ""

# comprobación de correcto funcionamiento
if auth_token is None:
    print("Authorization Token está vacio. Por favor, reinicie la aplicación después de configurar el token.")
else:
    print("Se ha encontrado un Authorization Token")

In [6]:
def getAPIrequest(auth_token, url):
    """
    Función para realizar peticiones GET a la API de Spotify.
    """
    response = requests.get(
            url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {auth_token}"
            }
        )
    return response

def postAPIrequest(auth_token, url, data):
    """
    Función para realizar peticiones POST a la API de Spotify.
    """
    response = requests.post(
           url,
           data=data,
           headers={
               "Content-Type": "application/json",
               "Authorization": f"Bearer {auth_token}"
           }
    )
    return response

## 2: Recuperar las canciones reproducidas recientemente por el usuario
Ahora, recuperamos las últimas _numOfTracks_ canciones reproducidas por el usuario. Para ello, enviamos una petición GET a la API de Spotify.

In [7]:
def getLastPlayedSongs(numOfTracks):
    """
    Función para obtener las últimas canciones reproducidas por el usuario.
    """
    url = f"https://api.spotify.com/v1/me/player/recently-played?limit={numOfTracks}"
    response = getAPIrequest(auth_token, url)
    response_json = response.json()
    songs = []
    #print(json.dumps(response_json, indent=4))
    try:
        for song in response_json["items"]:
            songs.append(song)
    except KeyError:
        print("Tu token de acceso a Spotify ha expirado.")
        print("Por favor, obtenga uno nuevo o pruebe otra vez.")
    return songs

In [ ]:
num = int(input("¿Cuántas pistas le gustaría visualizar? "))
lastPlayed = getLastPlayedSongs(num)
print(f"\nEstas son las últimas {num} canciones que has escuchado en Spotify:")
for index, track in enumerate(lastPlayed):
    print(f"\n {index+1}: {track['track']['name']}, {track['track']['artists'][0]['name']} ({track['track']['album']['release_date'][:4]})")

## 3: Obtener las preferencias del usuario

Ahora, pedimos al usuario que especifique las canciones que desea como base para sus recomendaciones. Introduzca la lista como una serie de índices separados por espacios. Por ejemplo, si quiere las canciones primera, tercera y quinta, introduzca 1 3 5

In [9]:
ref_tracks = input("\nIntroduzca una lista de hasta 5 pistas que se utilizarán como pistas iniciales: ") # introduzca los números separados por espacios de la pista
ref_tracks = ref_tracks.split()
seed_tracks = [lastPlayed[int(i)-1] for i in ref_tracks]
# print(seed_tracks)

## 4: Preprocesamiento de datos
Utilizando las opciones del usuario, las convertimos en una entrada adecuada con el modelo para que el formato coincida con el marco de datos que estamos utilizando.

In [ ]:
def get_song_info(song_list):
    """
    Función para obtener el nombre y el año de publicación de las pistas de canciones. 
    """
    seeds = []
    for item in range(len(song_list)):
        song = {'name': song_list[item]['track']['name'], 'artists': str([song_list[item]['track']['artists'][0]['name']]) }
        seeds.append(song)
    return seeds

get_song_info(seed_tracks)

## 5: Construcción de modelos y entrenamiento
Pasamos a construir y entrenar nuestro modelo. El código para hacerlo coincide exactamente con el código de Recommender.ipynb. 

In [ ]:
os.system("pip install numpy")
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

In [12]:
song_data = pd.read_csv('./data/data.csv')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=20, 
                                   verbose=2))],verbose=True)
X = song_data.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
np.save('my_history.npy',song_cluster_pipeline)
import joblib
joblib.dump(song_cluster_pipeline,'modelo_entrenado2.pkl')
song_cluster_labels = song_cluster_pipeline.predict(X)
song_data['cluster_label'] = song_cluster_labels

## 6: Realizar recomendaciones
Ahora procedemos a realizar recomendaciones basadas en las preferencias del usuario. El algoritmo seguido aquí es el mismo que el de Recommender.ipynb. Los únicos cambios son en las columnas de entrada y salida, para que estos datos sean aptos para la interacción con el endpoint de la API de Spotify. 

In [15]:
from collections import defaultdict
from scipy.spatial.distance import cdist
import difflib
    
def get_song_data(song, song_data):
    
    """
    Obtiene los datos de una canción específica. La canción adopta la forma de un diccionario con 
    pares clave-valor para el nombre y el año de lanzamiento.
    """
    try:
        song_info = song_data[(song_data['name'] == song['name']) 
                            & (song_data['artists'] == song['artists'])].iloc[0]
        return song_info
    except IndexError:
        return None

def get_mean_vector(song_list, song_data):
    """
   Obtiene el vector medio de una lista de canciones.
    """
    song_vectors = []
    for song in song_list:
        song_info = get_song_data(song, song_data)
        if song_info is None:
            print('Cuidado: {} no existe en la base de datos'.format(song['name']))
            continue
        song_vector = song_info[number_cols].values
        song_vectors.append(song_vector)  
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)

def flatten_dict_list(dict_list):
    """
    Función de utilidad para "flatering" una lista de diccionarios.
    """
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
    return flattened_dict

In [16]:
def recommend_songs(song_list, song_data, n_songs=12):
    """
   Recomienda canciones basándose en una lista de canciones anteriores que el usuario ha escuchado.
    """
    metadata_cols = ['name', 'year', 'artists', 'id']
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, song_data)
    scaler = song_cluster_pipeline.steps[0][1]
    scaled_data = scaler.transform(song_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = song_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols].to_dict(orient='records')[1:]

In [ ]:
recommended = recommend_songs(get_song_info(seed_tracks), song_data)
print(recommended)

## 7: Crear una lista de reproducción
Ahora creamos una lista de reproducción con las recomendaciones que hemos hallado.  

En primer lugar, creamos una lista de reproducción con un título especificado por el usuario realizando una solicitud POST a la API.
A continuación, comparamos el "id" de las canciones de la lista de canciones _recomendadas_ con sus respectivas URI de Spotify mediante una serie de peticiones GET.


Todos estos URIs se añaden a una lista JSON, que luego se utiliza en una solicitud POST a la API, para añadir estas canciones a la lista de reproducción que acabamos de crear.

In [18]:
playlist_name = input("Introduce el nombre de la playlist:")
playlist_description = "Playlist creada con un modelo de recomendaciones!"

In [19]:
def createPlaylist(name=playlist_name, description=playlist_description, user_id=user_id):
    """
    Esta función crea una lista de reproducción para el usuario con el nombre y la descripción dadas.
    """
    data = json.dumps({
            "name": name,
            "description": description,
            "public": True
        })
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    response = postAPIrequest(auth_token, url, data)
    response_json = response.json()
    playlist_id = response_json["id"]
    return playlist_id

In [20]:
def searchForTrack(track):
    """
    Esta función hace coincidir el id de una canción con su URI en Spotify.
    Los URIs son necesarios para añadir canciones a una lista de reproducción.
    """
    url = f"https://api.spotify.com/v1/tracks/{track['id']}"
    response = getAPIrequest(auth_token, url)
    response_json = response.json()
    track_uri = response_json["uri"]
    return track_uri
        

In [21]:
def addSongsToPlaylist(playlist_id, tracks):
    """
    Esta función encuentra los URI de todas las canciones recomendadas y las añade a la lista de reproducción.
    """
    track_uris = [searchForTrack(track) for track in tracks]
    #print(track_uris)
    data = json.dumps(track_uris)
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    response = postAPIrequest(auth_token, url, data)
    response_json = response.json()
    return response_json

In [ ]:
addSongsToPlaylist(createPlaylist(), recommended)

**Si la celda anterior devuelve un snapshot_id, significa que la lista de reproducción se ha creado con éxito. Por favor, comprueba tu cuenta de Spotify.**


# Transferencia de recomendaciones a Deezer

## 1: Crear una aplicación en Spotify

Debido a las restricciones de la API de Spotify, es necesario añadir el SpotifyForDevelopers a tu cuenta de Spotify para poder ejecutar este código. Ya que no es posible inicar sesión directamente en tu cuenta, con estos tokens podemos validar las credenciales y obterner la información desde Spotify.

### Pasos

1. Registra una aplicación Spotify (usando este enlace como guía https://developer.spotify.com/web-api/tutorial/).
2. Añade la URL http://localhost:8080/spotifyCallback en el apartado de Redirect URIs.
3. Modifique los valores del fichero "api-secrets.js" que se encuentra en la ruta Deezer/src/lib, con los datos del ID de cliente y la key de la aplicacion creada.

## 2: Crear una aplicación en Deezer

### Pasos

1. Registra una aplicación Deezer (usando este enlace como guía http://developers.deezer.com/guidelines/getting_started).
2. Ingresa la URL http://localhost:8080 en el apartado de Application domain, http://localhost:8080/deezerCallback en Redirect URL after authentication y un link aleatorio en Link to your Terms of Use.
3. Modifique los valores del fichero "api-secrets.js" que se encuentra en la ruta Deezer/src/lib, con los datos del ID de aplicacion y la key de la aplicacion creada.


## 3: Lanzar una interfaz en una ventana del navegador y realizar la transferencia
La API de Deezer nos obliga a seguir unos pasos para llevar a cabo la autententicación, estos estan ya estipulados y tienen que ser a través del navegador.

Por ello, existe una aplicación en la ruta Deezer/src, la cuál se lanzará a continuanción para poder seguir con el proceso.
Esta abrira un nueva ventana en la dirección http://localhost:8080, y alli se deberá realizar el inicio de sesión en Spotify, seleccionar la playlist que queremos transferir, que sera la creada anteriormente, y posteriormente iniciar sesión en Deezer y seleccionar la playlist de nuevo.

Despues de seguir estos pasos, la playlist se habrá transferido correctamente a la cuenta de Deezer

In [ ]:
import os 
import webbrowser

os.system("npm install ../Deezer ")
#Se instalan las dependencias necesarias 
webbrowser.open("http://localhost:8080", new=2, autoraise=True)
#Se lanza una nueva ventana con la URL donde se realizara la transferencia y que debera ser recargada
os.system("node ../Deezer/src/app.js")
# Se inicia la aplicacion
